In [2]:
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "drive/MyDrive/Informer-Tensorflow/models")
sys.path.insert(1, "drive/MyDrive/Informer-Tensorflow")
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

In [3]:
from attn import ProbAttention, AttentionLayer

In [4]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [5]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1)

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [6]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [25]:
def models():
    inputs = keras.layers.Input(shape=(127,15), batch_size=32)
    # x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = keras.layers.Conv1D(32, 1, activation='tanh')(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(128, 32)(x)
    x = ConvLayer(64)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(x)
    x = keras.layers.Dense(5)(x)
    return keras.Model(inputs, x)

In [26]:
model = models()
model.summary()
# del model

[TensorShape([32, 127, 32]), TensorShape([32, 127, 32]), TensorShape([32, 127, 32])]
[TensorShape([32, 42, 64]), TensorShape([32, 42, 64]), TensorShape([32, 42, 64])]
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(32, 127, 15)]           0         
                                                                 
 conv1d_12 (Conv1D)          (32, 127, 32)             512       
                                                                 
 positional_embedding_4 (Pos  (32, 127, 32)            0         
 itionalEmbedding)                                               
                                                                 
 multi_head_prob_attention_8  (32, 127, 128)           29184     
  (MultiHeadProbAttention)                                       
                                                                 
 conv_layer_8 (ConvLayer

In [27]:
x = tf.random.normal((32, 127, 15))
model(x)

<tf.Tensor: shape=(32, 5), dtype=float32, numpy=
array([[-3.15791624e-08, -9.93879894e-08,  1.85782874e-08,
        -2.39856002e-09,  1.18648220e-08],
       [-1.80651423e-08,  3.26347305e-08, -1.00061655e-07,
         4.39262537e-09, -2.52681378e-08],
       [ 1.62056040e-07, -7.18665127e-09,  9.79133752e-09,
         3.53844953e-08,  2.45822154e-08],
       [-2.43859297e-08,  4.57174139e-08, -4.50567006e-09,
         1.17765360e-07, -1.03416475e-08],
       [ 4.47322250e-08,  3.51199887e-08, -2.38543283e-08,
        -2.97572669e-08,  1.71023498e-08],
       [-1.77016048e-07, -8.41290131e-08,  1.88165874e-08,
        -7.36928030e-08,  2.71743001e-08],
       [-3.35577894e-08,  1.06789777e-08,  2.36702888e-08,
         5.56420687e-08, -7.44437898e-08],
       [-4.65531258e-08,  9.62876214e-08,  5.64138389e-08,
         1.24417326e-07, -3.52070941e-08],
       [-5.16893728e-09,  3.94202004e-08,  9.94779654e-08,
         1.08342405e-07,  2.84182491e-08],
       [-8.29361895e-08,  9.38712

In [19]:
data = np.load("mfcc.npz")
X = data["X"]
Y = data["Y"]



In [29]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [30]:
learning_rate = CustomSchedule(32)
optimizer = keras.optimizers.experimental.AdamW(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

# model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=["accuracy"])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./model_checkpoints', monitor='val_accuracy', save_best_only=True)


In [31]:


model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])


In [32]:
model.fit(x=X[0:832], y=Y[0:832], batch_size=32, epochs=1000, validation_data=(X[872:], Y[872:]),
         callbacks=[tensorboard_callback])


Epoch 1/1000
26/26 [==============================] - 11s 52ms/step - loss: 1.6094 - accuracy: 0.2260 - val_loss: 1.6094 - val_accuracy: 0.2031
Epoch 2/1000
26/26 [==============================] - 1s 26ms/step - loss: 1.6094 - accuracy: 0.2644 - val_loss: 1.6094 - val_accuracy: 0.2422
Epoch 3/1000
26/26 [==============================] - 1s 27ms/step - loss: 1.6093 - accuracy: 0.3065 - val_loss: 1.6093 - val_accuracy: 0.2656
Epoch 4/1000
26/26 [==============================] - 1s 27ms/step - loss: 1.6091 - accuracy: 0.3702 - val_loss: 1.6090 - val_accuracy: 0.2891
Epoch 5/1000
26/26 [==============================] - 1s 26ms/step - loss: 1.6085 - accuracy: 0.3558 - val_loss: 1.6081 - val_accuracy: 0.3438
Epoch 6/1000
26/26 [==============================] - 1s 27ms/step - loss: 1.6074 - accuracy: 0.4832 - val_loss: 1.6071 - val_accuracy: 0.2812
Epoch 7/1000
26/26 [==============================] - 1s 26ms/step - loss: 1.6051 - accuracy: 0.5264 - val_loss: 1.6044 - val_accuracy: 0.421